In [1]:
import clifford_opt as opt

In [2]:
import os
import sys
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import AerSimulator
import numpy as np
import pandas as pd

In [3]:
directory_path='QASMBench/'

In [4]:
def remove_conditions(circuit : QuantumCircuit):
    circ = QuantumCircuit(*circuit.cregs, *circuit.qregs)
    circ.name = circuit.name
    for instr, qargs, cargs in circuit.data:
        if (instr.condition is None):
            circ.append(instr, qargs, cargs)
    return circ

In [5]:
paths = []
for root, dirs, files in os.walk(directory_path):
    for dir_name in dirs:
        qasm_file_path = os.path.join(directory_path, dir_name, f"{dir_name}.qasm")
        number_at_end = int(''.join(filter(str.isdigit, dir_name)))
        if os.path.exists(qasm_file_path) and number_at_end <= 7:
            paths.append(qasm_file_path)

In [6]:
circuits = []
for i, path in enumerate(paths):    
    filename = path.split('/')[-1]
    sys.stdout.flush()
    circuit = QuantumCircuit.from_qasm_file(path)
    circuit.name = filename[:-5]
    circuit = remove_conditions(circuit)
    circuit.remove_final_measurements()
    circuit2 = transpile(circuit, optimization_level=0, basis_gates=opt.native_gates)
    circuit2.name = circuit.name
    circuits.append(circuit2)

In [7]:
def sorting_key(qc):
    return (qc.num_qubits, len(qc))

circuits_sorted = sorted(circuits, key=sorting_key)

In [24]:
data = []
shots = 10
for circuit in circuits_sorted:
    print('\\')
    print(circuit.name)
    shots = 10
    eps = 1
    fid1 = fidelity_density_matrix_orig(circuit, measure=True)
    while shots < 2000:
        epses = []
        k = 0
        while (k < 2):
            fid2 = fidelity_clifford_measuring(circuit, shots=shots, use_tqdm=False)
            data.append({'circuit' : circuit.name, 
                   'orig_fidelity' : fid1, 
                   'clifford_fidelity' : fid2, 
                    'shots': shots})
            eps = np.abs(fid2 - fid1) / fid1
            epses.append(eps)
            #print(shots, circuit.name, fid1, fid2, eps)
            print(shots, end= ',')
            k += 1
        eps = np.max(epses) if len(epses) else eps
        shots = int(shots * 1.3)

\
deutsch_n2
10,10,13,13,16,16,20,20,26,26,33,33,42,42,54,54,70,70,91,91,118,118,153,153,198,198,257,257,334,334,434,434,564,564,733,733,952,952,1237,1237,1608,1608,\
iswap_n2
10,10,13,13,16,16,20,20,26,26,33,33,42,42,54,54,70,70,91,91,118,118,153,153,198,198,257,257,334,334,434,434,564,564,733,733,952,952,1237,1237,1608,1608,\
grover_n2
10,10,13,13,16,16,20,20,26,26,33,33,42,42,54,54,70,70,91,91,118,118,153,153,198,198,257,257,334,334,434,434,564,564,733,733,952,952,1237,1237,1608,1608,\
quantumwalks_n2
10,10,13,13,16,16,20,20,26,26,33,33,42,42,54,54,70,70,91,91,118,118,153,153,198,198,257,257,334,334,434,434,564,564,733,733,952,952,1237,1237,1608,1608,\
ipea_n2
10,10,13,13,16,16,20,20,26,26,33,33,42,42,54,54,70,70,91,91,118,118,153,153,198,198,257,257,334,334,434,434,564,564,733,733,952,952,1237,1237,1608,1608,\
dnn_n2
10,10,13,13,16,16,20,20,26,26,33,33,42,42,54,54,70,70,91,91,118,118,153,153,198,198,257,257,334,334,434,434,564,564,733,733,952,952,1237,1237,1608,1608,\
teleportation

In [30]:
df = pd.DataFrame(data)
df['qubit_count'] = df['circuit'].apply(lambda c: int(c[-1]))
df['eps'] = np.abs(df['orig_fidelity'] - df['clifford_fidelity']) / df['orig_fidelity']
lens = {c.name : len(c) for c in circuits_sorted}
circs = {c.name : c for c in circuits_sorted}
def peaks(c1 : QuantumCircuit):
    c = c1.copy()
    g = AerSimulator()
    c.measure_all()
    return len(g.run(c, shots=2000).result().data()['counts'])
peakss = {c.name: peaks(c) for c in circuits_sorted}
df['len'] = df['circuit'].apply(lambda c: lens[c])
df['peaks'] = df['circuit'].apply(lambda c: peakss[c])

In [9]:
opt.transpiler(circuits_sorted[5], use_tqdm=True, shots=1000, func=opt.Qfidelity_clifford_measuring)

0.5039677419353517 -18.774582046646
0.5651198187994889 -15.55058204664601
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.55058204664598
0.5651198187994889 -15.55058204664598
0.5651198187994889 -15.55058204664598
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.55058204664598
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.55058204664598
0.5651198187994889 -15.55058204664598
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.55058204664598
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.55058204664598
0.5651198187994889 -15.55058204664598
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.55058204664598
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.550582046645982
0.5651198187994889 -15.550582046645982


KeyboardInterrupt: 

In [17]:
res_c = transpiler(circuits_sorted[12], use_tqdm=True, shots=1000, n_iter=10)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 319.83it/s]


0.129 -4.093459738486393


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 301.90it/s]


0.146 -4.093459738486393


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 330.68it/s]


...

0.16100000000000003 -4.025793071819726


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 329.77it/s]


0.16100000000000003 -4.025793071819726


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 308.57it/s]


0.165 -4.025793071819726


In [ ]:
opt.transpiler()